## Set up

In [1]:
!pip install transformers==4.37.0
!pip install datasets

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 129.4/129.4 kB 4.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.4/8.4 MB 15.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.6/3.6 MB 26.2 MB/s eta 0:00:00
  Attempting uninstall: tokenizers
    Found existing installation: tokenizers 0.20.3
    Uninstalling tokenizers-0.20.3:
      Successfully uninstalled tokenizers-0.20.3
  Attempting uninstall: transformers
    Found existing installation: transformers 4.46.2
    Uninstalling transformers-4.46.2:
      Successfully uninstalled transformers-4.46.2
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
sentence-transformers 3.2.1 requires transformers<5.0.0,>=4.41.0, but you have transformers 4.37.0 which is incompatible.
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 480.6/480.6 kB 14.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [2]:
import torch
import pandas as pd
from sklearn.model_selection import train_test_split
from torch.utils.data import Dataset, TensorDataset, DataLoader
from torch.nn import CrossEntropyLoss
from torch.nn.utils.rnn import pad_sequence
from tqdm import tqdm
import numpy as np
from transformers import AutoTokenizer, AutoModelForSequenceClassification
from transformers import AdamW
import os
from sklearn.metrics import classification_report, precision_score, recall_score, f1_score
from datasets import load_dataset

import csv

torch.manual_seed(42)
np.random.seed(42)

## Google Drive linking
Make sure you have a CSE_354_Project in your Google Drive


In [3]:
%cd /content/drive/MyDrive/CSE_354_Project/

[Errno 2] No such file or directory: '/content/drive/MyDrive/CSE_354_Project/'
/content


In [4]:
from google.colab import drive
drive.mount('/content/drive')
base_dir = "/content/drive/MyDrive/CSE_354_Project/"
%cd $base_dir

Mounted at /content/drive
/content/drive/MyDrive/CSE_354_Project


## Loading the dataset
This make an API request to hugging face then we verify the data

In [5]:
train_dataset = load_dataset("community-datasets/per_sent", split="train")
test_random_dataset = load_dataset("community-datasets/per_sent", split="test_random")
test_fixed_dataset = load_dataset("community-datasets/per_sent", split="test_fixed")
validation_dataset = load_dataset("community-datasets/per_sent", split="validation")

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


README.md:   0%|          | 0.00/14.0k [00:00<?, ?B/s]

train-00000-of-00001.parquet:   0%|          | 0.00/8.84M [00:00<?, ?B/s]

test_random-00000-of-00001.parquet:   0%|          | 0.00/1.60M [00:00<?, ?B/s]

test_fixed-00000-of-00001.parquet:   0%|          | 0.00/2.27M [00:00<?, ?B/s]

validation-00000-of-00001.parquet:   0%|          | 0.00/1.41M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/3355 [00:00<?, ? examples/s]

Generating test_random split:   0%|          | 0/579 [00:00<?, ? examples/s]

Generating test_fixed split:   0%|          | 0/827 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/578 [00:00<?, ? examples/s]

## IMPORTANT NOTE ON RUNNING THE REST OF THE CODES

The following is a diversion, there's 4 different ways to obtain a model
1. Baseline
2. Idea 1 (TGT tokens)
3. Idea 2 (Paragraph + Document)
  - include Idea 2.1 and Idea 2.2
4. Idea 3 (Weighted class)

To obtain a specific model, run only that specific way codes and skip others.

## Baseline Model Creation
Using DistillBERT with 3 classes, and a special mask_token which is represented by [TGT] in Masked document of the dataset

In [ ]:
class DistillBERT():
  def __init__(self, model_name='distilbert-base-uncased', num_classes=3):
    self.tokenizer = AutoTokenizer.from_pretrained(model_name)
    self.tokenizer.add_special_tokens({'mask_token': '[TGT]'})
    self.model = AutoModelForSequenceClassification.from_pretrained(model_name, num_labels=num_classes)
    self.model.resize_token_embeddings(len(self.tokenizer))


  def get_tokenizer_and_model(self):
    return self.model, self.tokenizer

Using the model's tokenizer to tokenize the input dataset

In [ ]:
class DatasetLoader(Dataset):

  def __init__(self, dataset, tokenizer):
    self.dataset = dataset
    self.tokenizer = tokenizer

  def tokenize_data(self):
    print("Tokenizing data...")
    tokens = []
    labels = []

    for row in tqdm(self.dataset):
      text = row['MASKED_DOCUMENT']
      label = row['TRUE_SENTIMENT']

      tokens.append(torch.tensor(self.tokenizer.encode(text, max_length=512, truncation=True, add_special_tokens=True)))
      labels.append(label)


    tokens = pad_sequence(tokens, batch_first=True)
    labels = torch.tensor(labels)
    dataset = TensorDataset(tokens, labels)
    return dataset

  def get_data_loaders(self, batch_size=32, shuffle=True):
    processed_dataset = self.tokenize_data()

    data_loader = DataLoader(
        processed_dataset,
        shuffle=shuffle,
        batch_size=batch_size
    )

    return data_loader

The model's fine-tuning stages by giving it the task specific data.

The Optimizer is created in the execution stage, which have the hyperparameters of learning rate and epsilon.

In [ ]:
class Trainer():

  def __init__(self, options):
    self.device = options['device']
    self.train_data = options['train_data']
    self.batch_size = options['batch_size']
    self.epochs = options['epochs']
    self.save_path = options['save_path']
    transformer = DistillBERT()
    self.model, self.tokenizer = transformer.get_tokenizer_and_model()
    self.model.to(self.device)

  def get_performance_metrics(self, preds, labels):
    pred_flat = np.argmax(preds, axis=1).flatten()
    labels_flat = labels.flatten()
    precision = precision_score(labels_flat, pred_flat, zero_division=0, average='weighted')
    recall = recall_score(labels_flat, pred_flat, zero_division=0, average='weighted')
    f1 = f1_score(labels_flat, pred_flat, zero_division=0, average='weighted')
    return precision, recall, f1

  def set_training_parameters(self):
    for i, layer in enumerate(self.model.distilbert.transformer.layer):
      for param in layer.parameters():
        param.requires_grad = True

    for param in self.model.classifier.parameters():
      param.requires_grad = True

  def train(self, data_loader, optimizer):
    self.model.train()
    total_recall = 0
    total_precision = 0
    total_f1 = 0
    total_loss = 0

    for batch_idx, (text, labels) in enumerate(tqdm(data_loader)):
      self.model.zero_grad()

      text = text.to(self.device)
      labels = labels.to(self.device)

      outputs = self.model(text, labels=labels)
      predicted = outputs.logits.cpu().detach().numpy()

      loss = outputs.loss
      total_loss += loss.item()
      loss.backward()
      optimizer.step()

      precision, recall, f1 = self.get_performance_metrics(predicted, labels.cpu().numpy())
      total_precision += precision
      total_recall += recall
      total_f1 += f1

    precision = total_precision/len(data_loader)
    recall = total_recall/len(data_loader)
    f1 = total_f1/len(data_loader)
    loss = total_loss/len(data_loader)

    return precision, recall, f1, loss

  def eval(self, data_loader):
    self.model.eval()
    total_recall = 0
    total_precision = 0
    total_f1 = 0
    total_loss = 0

    with torch.no_grad():
      for batch_idx, (text, labels) in enumerate(tqdm(data_loader)):
        text = text.to(self.device)
        labels = labels.to(self.device)

        outputs = self.model(text, labels=labels)
        predicted = outputs.logits.cpu().detach().numpy()

        loss = outputs.loss
        total_loss += loss.item()

        precision, recall, f1 = self.get_performance_metrics(predicted, labels.cpu().numpy())
        total_precision += precision
        total_recall += recall
        total_f1 += f1

      precision = total_precision/len(data_loader)
      recall = total_recall/len(data_loader)
      f1 = total_f1/len(data_loader)
      loss = total_loss/len(data_loader)

      return precision, recall, f1, loss

  def save_transformer(self):
    self.model.save_pretrained(self.save_path)
    self.tokenizer.save_pretrained(self.save_path)

  def execute(self):
    last_best = 0
    train_dataset = DatasetLoader(self.train_data, self.tokenizer)
    train_data_loader = train_dataset.get_data_loaders(self.batch_size)

    val_dataset = DatasetLoader(validation_dataset, self.tokenizer)
    val_data_loader = val_dataset.get_data_loaders(self.batch_size)

    optimizer = torch.optim.AdamW(self.model.parameters(), lr = 3e-5, eps = 1e-8)
    self.set_training_parameters()

    for epoch_i in range(0, self.epochs):
      train_precision, train_recall, train_f1, train_loss = self.train(train_data_loader, optimizer)
      print(f'Epoch {epoch_i + 1}: train_loss: {train_loss:.4f} train_precision: {train_precision:.4f} train_recall: {train_recall:.4f} train_f1: {train_f1:.4f}')
      val_precision, val_recall, val_f1, val_loss = self.eval(val_data_loader)
      print(f'Epoch {epoch_i + 1}: val_loss: {val_loss:.4f} val_precision: {val_precision:.4f} val_recall: {val_recall:.4f} val_f1: {val_f1:.4f}')

      if val_f1 > last_best:
        print("Saving model..")
        self.save_transformer()
        last_best = val_f1
        print("Model saved.")

Hyper Params of
1. Batch size
2. Epochs

Then the additional of save path of the model which is set to *distilbert_per_sent_baseline* at the moment

In [ ]:
BATCH_SIZE = 16
EPOCHS = 3

SAVE_PATH = 'distilbert_per_sent_baseline'

## Baseline Training
training on the train_dataset we've loaded up previously, then verfying it with the validation_dataset each epochs to see the improvement of the model

In [ ]:
options = {}
options['device'] = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
options['train_data'] = train_dataset
options['batch_size'] = BATCH_SIZE
options['epochs'] = EPOCHS
options['save_path'] = SAVE_PATH

trainer = Trainer(options)
trainer.execute()

/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:797: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/483 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

...

Epoch 2: val_loss: 0.8694 val_precision: 0.5858 val_recall: 0.5760 val_f1: 0.5439


100%|██████████| 210/210 [02:38<00:00,  1.32it/s]


Epoch 3: train_loss: 0.7192 train_precision: 0.6709 train_recall: 0.6720 train_f1: 0.6489


100%|██████████| 37/37 [00:09<00:00,  3.94it/s]


Epoch 3: val_loss: 0.9075 val_precision: 0.6223 val_recall: 0.5777 val_f1: 0.5762
Saving model..
Model saved.


## Baseline Testing
Verifying the model performance with fresh data from testing split.

In [ ]:
class Tester:
  def __init__(self, options):
    self.device = options['device']
    self.test_data = options['test_data']
    self.batch_size = options['batch_size']
    self.save_path = options['save_path']

    transformer = DistillBERT()
    self.model, self.tokenizer = transformer.get_tokenizer_and_model()
    self.model.to(self.device)

  def load_transformer(self):
    self.model = AutoModelForSequenceClassification.from_pretrained(self.save_path)
    self.tokenizer = AutoTokenizer.from_pretrained(self.save_path)
    self.model.to(self.device)
    self.model.eval()

  def get_performance_metrics(self, preds, labels):
    pred_flat = np.argmax(preds, axis=1).flatten()
    labels_flat = labels.flatten()
    precision = precision_score(labels_flat, pred_flat, zero_division=0, average='weighted')
    recall = recall_score(labels_flat, pred_flat, zero_division=0, average='weighted')
    f1 = f1_score(labels_flat, pred_flat, zero_division=0, average='weighted')
    return precision, recall, f1

  def test(self, data_loader):
    self.model.eval()
    total_recall = 0
    total_precision = 0
    total_f1 = 0
    total_loss = 0

    y_true = []
    y_pred = []

    with torch.no_grad():
      for batch_idx, (text, labels) in enumerate(tqdm(data_loader)):
        text = text.to(self.device)
        labels = labels.to(self.device)

        outputs = self.model(text, labels=labels)
        predicted = outputs.logits.cpu().detach().numpy()
        loss = outputs.loss
        total_loss += loss.item()

        labels = labels.cpu().numpy()
        y_true.extend(labels)
        y_pred.extend(np.argmax(predicted, axis=1))

        precision, recall, f1 = self.get_performance_metrics(predicted, labels)
        total_precision += precision
        total_recall += recall
        total_f1 += f1

      precision = total_precision / len(data_loader)
      recall = total_recall / len(data_loader)
      f1 = total_f1 / len(data_loader)
      loss = total_loss / len(data_loader)
      report = classification_report(y_true, y_pred, target_names=["Negative", "Neutral", "Positive"])

      return precision, recall, f1, loss, report

  def execute(self):
    self.load_transformer()
    test_dataset = DatasetLoader(self.test_data, self.tokenizer)
    test_data_loader = test_dataset.get_data_loaders(self.batch_size)
    precision, recall, f1, loss, report = self.test(test_data_loader)
    print(f'precision: {precision:.4f} recall: {recall:.4f} f1: {f1:.4f}')
    print(f'Test loss: {loss:.4f}')
    print()
    print(report)


Testing on the test_random_dataset (random and different main target entity)

In [ ]:
options={}
options['device'] = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
options['test_data'] = test_random_dataset
options['batch_size'] = BATCH_SIZE
options['save_path'] = SAVE_PATH

tester = Tester(options)
tester.execute()

/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:797: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight', 'pre_classifier.bias', 'pre_classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Tokenizing data...


100%|██████████| 37/37 [00:09<00:00,  3.93it/s]

precision: 0.5998 recall: 0.5552 f1: 0.5515
Test loss: 0.9371

              precision    recall  f1-score   support

    Negative       0.32      0.44      0.37        73
     Neutral       0.48      0.39      0.43       213
    Positive       0.68      0.70      0.69       293

    accuracy                           0.55       579
   macro avg       0.49      0.51      0.50       579
weighted avg       0.56      0.55      0.55       579



Testing on the test_fixed_dataset (stuff have repeat high frequence main target entity)

In [ ]:
options={}
options['device'] = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
options['test_data'] = test_fixed_dataset
options['batch_size'] = BATCH_SIZE
options['save_path'] = SAVE_PATH

tester = Tester(options)
tester.execute()

/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:797: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight', 'pre_classifier.bias', 'pre_classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Tokenizing data...


100%|██████████| 52/52 [00:13<00:00,  3.76it/s]

precision: 0.5082 recall: 0.4768 f1: 0.4652
Test loss: 1.1159

              precision    recall  f1-score   support

    Negative       0.33      0.26      0.29       139
     Neutral       0.45      0.38      0.41       320
    Positive       0.53      0.64      0.58       368

    accuracy                           0.48       827
   macro avg       0.44      0.43      0.43       827
weighted avg       0.46      0.48      0.47       827



---
## Idea 1 Mask TGT Token for Classification
Below are the implementation for idea 1 (it will redefined the classes in Baseline)

Using DistillBERT with 3 classes, and a special mask_token which is represented by [TGT] in Masked document of the dataset

In [9]:
class DistillBERT():
  def __init__(self, model_name='distilbert-base-uncased', num_classes=3):
    self.tokenizer = AutoTokenizer.from_pretrained(model_name)
    self.tokenizer.add_special_tokens({'mask_token': '[TGT]'})
    self.model = AutoModelForSequenceClassification.from_pretrained(model_name, num_labels=num_classes)
    self.model.resize_token_embeddings(len(self.tokenizer))


  def get_tokenizer_and_model(self):
    return self.model, self.tokenizer

Using the model's tokenizer to tokenize the input dataset

In [8]:
class DatasetLoader(Dataset):

  def __init__(self, dataset, tokenizer):
    self.dataset = dataset
    self.tokenizer = tokenizer

  def tokenize_data(self):
    print("Tokenizing data...")
    tokens = []
    labels = []

    data_not_have_tgt = 0
    data_does_not_have_tgt_before_limit = 0
    tgt_token_id = self.tokenizer.convert_tokens_to_ids("[TGT]")


    for row in tqdm(self.dataset):
      text = row['MASKED_DOCUMENT']
      label = row['TRUE_SENTIMENT']

      if '[TGT]' not in text:
        data_not_have_tgt += 1
        continue

      token = self.tokenizer.encode(text, max_length=512, truncation=True, add_special_tokens=True)

      if tgt_token_id not in token:
        data_does_not_have_tgt_before_limit += 1
        continue

      tokens.append(torch.tensor(token))
      labels.append(label)


    tokens = pad_sequence(tokens, batch_first=True)
    labels = torch.tensor(labels)
    dataset = TensorDataset(tokens, labels)
    print("Data thrown away: " + str(data_not_have_tgt) + ", Reason: due to not having [TGT] tokens")
    print("Data thrown away: " + str(data_does_not_have_tgt_before_limit) + ", Reason: due to not having [TGT] tokens before limit")
    return dataset

  def get_data_loaders(self, batch_size=32, shuffle=True):
    processed_dataset = self.tokenize_data()

    data_loader = DataLoader(
        processed_dataset,
        shuffle=shuffle,
        batch_size=batch_size
    )

    return data_loader

The model's fine-tuning stages by giving it the task specific data.

The Optimizer is created in the execution stage, which have the hyperparameters of learning rate and epsilon.

In [6]:
class Trainer():

  def __init__(self, options):
    self.device = options['device']
    self.train_data = options['train_data']
    self.batch_size = options['batch_size']
    self.epochs = options['epochs']
    self.save_path = options['save_path']
    transformer = DistillBERT()
    self.model, self.tokenizer = transformer.get_tokenizer_and_model()
    self.model.to(self.device)

  def get_performance_metrics(self, preds, labels):
    pred_flat = np.argmax(preds, axis=1).flatten()
    labels_flat = labels.flatten()
    precision = precision_score(labels_flat, pred_flat, zero_division=0, average='weighted')
    recall = recall_score(labels_flat, pred_flat, zero_division=0, average='weighted')
    f1 = f1_score(labels_flat, pred_flat, zero_division=0, average='weighted')
    return precision, recall, f1

  def set_training_parameters(self):
    for i, layer in enumerate(self.model.distilbert.transformer.layer):
      for param in layer.parameters():
        param.requires_grad = True

    for param in self.model.classifier.parameters():
      param.requires_grad = True

  def train(self, data_loader, optimizer):
    self.model.train()
    total_recall = 0
    total_precision = 0
    total_f1 = 0
    total_loss = 0

    for batch_idx, (text, labels) in enumerate(tqdm(data_loader)):
      self.model.zero_grad()

      text = text.to(self.device)
      labels = labels.to(self.device)

      outputs = self.model(text, labels=labels, output_hidden_states=True)
      last_layer = outputs.hidden_states[-1]

      tgt_token_id = self.tokenizer.convert_tokens_to_ids("[TGT]")
      tgt_mask = (text == tgt_token_id).unsqueeze(-1)

      # Commented out since was running into issue with the values, not needed anymore
      # issue was document 512 token does not have [TGT] within the limit but outside the limit

      # if (tgt_mask.sum(dim=1) == 0).any():
      #   print(f"Skipping batch {batch_idx} because [TGT] is missing.")

      #   missing_tgt_mask = (tgt_mask.sum(dim=1) == 0).squeeze(-1)
      #   missing_indices = missing_tgt_mask.nonzero(as_tuple=True)[0]

      #   for idx in missing_indices:
      #     decoded_text = self.tokenizer.decode(text[idx].tolist(), skip_special_tokens=False)
      #     print(f"Decoded text for row {idx}: {decoded_text}")

      #   continue

      masked_last_layer = last_layer * tgt_mask
      tgt_embeddings = masked_last_layer.sum(dim=1) / (tgt_mask.sum(dim=1))

      logits = self.model.classifier(tgt_embeddings)
      loss = CrossEntropyLoss()(logits, labels)

      total_loss += loss.item()

      loss.backward()
      optimizer.step()

      predicted = logits.cpu().detach().numpy()
      precision, recall, f1 = self.get_performance_metrics(predicted, labels.cpu().numpy())
      total_precision += precision
      total_recall += recall
      total_f1 += f1


    precision = total_precision/len(data_loader)
    recall = total_recall/len(data_loader)
    f1 = total_f1/len(data_loader)
    loss = total_loss/len(data_loader)

    return precision, recall, f1, loss

  def eval(self, data_loader):
    self.model.eval()
    total_recall = 0
    total_precision = 0
    total_f1 = 0
    total_loss = 0

    with torch.no_grad():
      for batch_idx, (text, labels) in enumerate(tqdm(data_loader)):
        text = text.to(self.device)
        labels = labels.to(self.device)

        outputs = self.model(text, labels=labels, output_hidden_states=True)
        last_layer = outputs.hidden_states[-1]

        tgt_token_id = self.tokenizer.convert_tokens_to_ids("[TGT]")
        tgt_mask = (text == tgt_token_id).unsqueeze(-1)

        # Commented out since was running into issue with the values, not needed anymore
        # issue was document 512 token does not have [TGT] within the limit but outside the limit

        # if (tgt_mask.sum(dim=1) == 0).any():
        #   print(f"Skipping batch {batch_idx} because [TGT] is missing.")
        #   continue

        masked_last_layer = last_layer * tgt_mask
        tgt_embeddings = masked_last_layer.sum(dim=1) / (tgt_mask.sum(dim=1))

        logits = self.model.classifier(tgt_embeddings)
        loss = CrossEntropyLoss()(logits, labels)

        total_loss += loss.item()

        predicted = logits.cpu().detach().numpy()
        precision, recall, f1 = self.get_performance_metrics(predicted, labels.cpu().numpy())
        total_precision += precision
        total_recall += recall
        total_f1 += f1

      precision = total_precision/len(data_loader)
      recall = total_recall/len(data_loader)
      f1 = total_f1/len(data_loader)
      loss = total_loss/len(data_loader)

      return precision, recall, f1, loss

  def save_transformer(self):
    self.model.save_pretrained(self.save_path)
    self.tokenizer.save_pretrained(self.save_path)

  def execute(self):
    last_best = 0
    train_dataset = DatasetLoader(self.train_data, self.tokenizer)
    train_data_loader = train_dataset.get_data_loaders(self.batch_size)

    val_dataset = DatasetLoader(validation_dataset, self.tokenizer)
    val_data_loader = val_dataset.get_data_loaders(self.batch_size)

    optimizer = torch.optim.AdamW(self.model.parameters(), lr = 3e-5, eps = 1e-8)
    self.set_training_parameters()

    for epoch_i in range(0, self.epochs):
      train_precision, train_recall, train_f1, train_loss = self.train(train_data_loader, optimizer)
      print(f'Epoch {epoch_i + 1}: train_loss: {train_loss:.4f} train_precision: {train_precision:.4f} train_recall: {train_recall:.4f} train_f1: {train_f1:.4f}')

      val_precision, val_recall, val_f1, val_loss = self.eval(val_data_loader)
      print(f'Epoch {epoch_i + 1}: val_loss: {val_loss:.4f} val_precision: {val_precision:.4f} val_recall: {val_recall:.4f} val_f1: {val_f1:.4f}')

      if val_f1 > last_best:
        print("Saving model..")
        self.save_transformer()
        last_best = val_f1
        print("Model saved.")

Hyper Params of
1. Batch size
2. Epochs

Then the additional of save path of the model which is set to *distilbert_per_sent_idea_1* at the moment

In [10]:
BATCH_SIZE = 16
EPOCHS = 3

SAVE_PATH = 'distilbert_per_sent_idea_1'

## Idea 1 Training

training on the train_dataset we've loaded up previously, then verfying it with the validation_dataset each epochs to see the improvement of the model

In [ ]:
options = {}
options['device'] = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
options['train_data'] = train_dataset
options['batch_size'] = BATCH_SIZE
options['epochs'] = EPOCHS
options['save_path'] = SAVE_PATH

trainer = Trainer(options)
trainer.execute()

torch.cuda.empty_cache()

/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:797: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight', 'pre_classifier.bias', 'pre_classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Tokenizing data...


100%|██████████| 3355/3355 [00:05<00:00, 575.64it/s]


Data thrown away: 142, Reason: due to not having [TGT] tokens
Data thrown away: 28, Reason: due to not having [TGT] tokens before limit
Tokenizing data...


100%|██████████| 578/578 [00:01<00:00, 520.22it/s]


...

Epoch 2: val_loss: 0.8372 val_precision: 0.5753 val_recall: 0.5735 val_f1: 0.5508
Saving model..
Model saved.


100%|██████████| 200/200 [02:48<00:00,  1.18it/s]


Epoch 3: train_loss: 0.6856 train_precision: 0.6931 train_recall: 0.6747 train_f1: 0.6575


100%|██████████| 34/34 [00:10<00:00,  3.25it/s]


Epoch 3: val_loss: 0.8639 val_precision: 0.6047 val_recall: 0.5993 val_f1: 0.5897
Saving model..
Model saved.


## Idea 1 Testing

Verify the model performance with fresh data from testing split

In [13]:
class Tester:
  def __init__(self, options):
    self.device = options['device']
    self.test_data = options['test_data']
    self.batch_size = options['batch_size']
    self.save_path = options['save_path']
    self.output_file = options['output_file']

    transformer = DistillBERT()
    self.model, self.tokenizer = transformer.get_tokenizer_and_model()
    self.model.to(self.device)

  def load_transformer(self):
    self.model = AutoModelForSequenceClassification.from_pretrained(self.save_path)
    self.tokenizer = AutoTokenizer.from_pretrained(self.save_path)
    self.model.to(self.device)
    self.model.eval()

  def get_performance_metrics(self, preds, labels):
    pred_flat = np.argmax(preds, axis=1).flatten()
    labels_flat = labels.flatten()
    precision = precision_score(labels_flat, pred_flat, zero_division=0, average='weighted')
    recall = recall_score(labels_flat, pred_flat, zero_division=0, average='weighted')
    f1 = f1_score(labels_flat, pred_flat, zero_division=0, average='weighted')
    return precision, recall, f1

  def test(self, data_loader):
    self.model.eval()
    total_recall = 0
    total_precision = 0
    total_f1 = 0
    total_loss = 0

    y_true = []
    y_pred = []

    failed_cases = []

    with torch.no_grad():
      for batch_idx, (text, labels) in enumerate(tqdm(data_loader)):
        text = text.to(self.device)
        labels = labels.to(self.device)

        outputs = self.model(text, labels=labels, output_hidden_states=True)
        last_layer = outputs.hidden_states[-1]

        tgt_token_id = self.tokenizer.convert_tokens_to_ids("[TGT]")
        tgt_mask = (text == tgt_token_id).unsqueeze(-1)

        masked_last_layer = last_layer * tgt_mask
        tgt_embeddings = masked_last_layer.sum(dim=1) / (tgt_mask.sum(dim=1))

        logits = self.model.classifier(tgt_embeddings)
        loss = CrossEntropyLoss()(logits, labels)

        total_loss += loss.item()

        predicted = logits.cpu().detach().numpy()
        labels = labels.cpu().numpy()
        precision, recall, f1 = self.get_performance_metrics(predicted, labels)
        y_true.extend(labels)
        predicted = np.argmax(predicted, axis=1)
        y_pred.extend(predicted)

        mismatches = np.where(labels != predicted)[0]
        for idx in mismatches:
            failed_cases.append((self.tokenizer.decode(text[idx].cpu().numpy()), labels[idx], predicted[idx]))

        total_precision += precision
        total_recall += recall
        total_f1 += f1

      precision = total_precision/len(data_loader)
      recall = total_recall/len(data_loader)
      f1 = total_f1/len(data_loader)
      loss = total_loss/len(data_loader)
      print()
      report = classification_report(y_true, y_pred, target_names=["Negative", "Neutral", "Positive"])

      with open(self.output_file, 'w', newline='') as file:
        writer = csv.writer(file)
        writer.writerow(['Text', 'True Label', 'Predicted Label'])
        writer.writerows(failed_cases)

      return precision, recall, f1, loss, report

  def execute(self):
    self.load_transformer()
    test_dataset = DatasetLoader(self.test_data, self.tokenizer)
    test_data_loader = test_dataset.get_data_loaders(self.batch_size)
    precision, recall, f1, loss, report = self.test(test_data_loader)
    print(f'precision: {precision:.4f} recall: {recall:.4f} f1: {f1:.4f}')
    print(f'Test loss: {loss:.4f}')
    print()
    print(report)


Testing on the test_random_dataset (random and different main target entity)

In [14]:
options={}
options['device'] = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
options['test_data'] = test_random_dataset
options['batch_size'] = BATCH_SIZE
options['save_path'] = SAVE_PATH
options['output_file'] = 'idea1_fail_test_random.csv'

tester = Tester(options)
tester.execute()

torch.cuda.empty_cache()

/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:797: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight', 'pre_classifier.bias', 'pre_classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Tokenizing data...


100%|██████████| 579/579 [00:03<00:00, 183.43it/s]


Data thrown away: 26, Reason: due to not having [TGT] tokens
Data thrown away: 10, Reason: due to not having [TGT] tokens before limit


100%|██████████| 34/34 [00:09<00:00,  3.72it/s]


precision: 0.5882 recall: 0.5768 f1: 0.5613
Test loss: 0.9189

              precision    recall  f1-score   support

    Negative       0.33      0.09      0.15        64
     Neutral       0.47      0.63      0.54       203
    Positive       0.70      0.65      0.68       276

    accuracy                           0.58       543
   macro avg       0.50      0.46      0.45       543
weighted avg       0.57      0.58      0.56       543



Testing on the test_fixed_dataset (stuff have repeat high frequence main target entity)

In [16]:
options={}
options['device'] = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
options['test_data'] = test_fixed_dataset
options['batch_size'] = BATCH_SIZE
options['save_path'] = SAVE_PATH
options['output_file'] = 'idea1_failed_test_fixed.csv'

tester = Tester(options)
tester.execute()

torch.cuda.empty_cache()

/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:797: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight', 'pre_classifier.bias', 'pre_classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Tokenizing data...


100%|██████████| 827/827 [00:05<00:00, 154.01it/s]


Data thrown away: 14, Reason: due to not having [TGT] tokens
Data thrown away: 5, Reason: due to not having [TGT] tokens before limit


100%|██████████| 51/51 [00:14<00:00,  3.63it/s]


precision: 0.4772 recall: 0.4571 f1: 0.4446
Test loss: 1.1037

              precision    recall  f1-score   support

    Negative       0.27      0.16      0.20       133
     Neutral       0.42      0.51      0.46       311
    Positive       0.54      0.53      0.53       364

    accuracy                           0.46       808
   macro avg       0.41      0.40      0.40       808
weighted avg       0.45      0.46      0.45       808



## Idea 2 Paragraph + Document level training

Below are the implementation for idea 2 (it will redefined the classes in Baseline)

Using DistillBERT with 3 classes, and a special mask_token which is represented by [TGT] in Masked document of the dataset

In [ ]:
class DistillBERT():
  def __init__(self, model_name='distilbert-base-uncased', num_classes=3):
    self.tokenizer = AutoTokenizer.from_pretrained(model_name)
    self.tokenizer.add_special_tokens({'mask_token': '[TGT]'})
    self.model = AutoModelForSequenceClassification.from_pretrained(model_name, num_labels=num_classes)
    self.model.resize_token_embeddings(len(self.tokenizer))


  def get_tokenizer_and_model(self):
    return self.model, self.tokenizer

Using the model's tokenizer to tokenize the input dataset

(which we find paragraph level by splitting by \n)

In [ ]:
class ParagraphDatasetLoader(Dataset):
  def __init__(self, dataset, tokenizer):
      self.dataset = dataset
      self.tokenizer = tokenizer

  def tokenize_data(self):
      print("Tokenizing data...")
      tokens = []
      paragraphs = []
      all_labels = []
      for row in tqdm(self.dataset):
        #1st, 2nd document in str
        text = row['MASKED_DOCUMENT']
        #true sentimenet of whole articles
        text_label = row['TRUE_SENTIMENT']

        paragraph_labels = []
        labels = [label for label in row.keys() if label.startswith('Paragraph')]
        paragraphs = text.split('\n')

        paragraphs.append(text)
        #tokenize and adding label for each paragraph
        for i,paragraph in zip(range(len(paragraphs)),paragraphs):
          #last "paragraph" is the whole text
          if i == len(paragraphs) - 1:
            tokens.append(torch.tensor(self.tokenizer.encode(paragraph, max_length=512, truncation=True, add_special_tokens=True)))
            all_labels.append(text_label)
          # #for every other paragraph
          else:
            tokens.append(torch.tensor(self.tokenizer.encode(paragraph, max_length=512, truncation=True, add_special_tokens=True)))
            #surpass 15 paragraphs use the  document label
            if i > 15:
              all_labels.append(text_label)
            else:
              #if there's a paragraph label(0,1,2) assign that to corresponding paragraph, ow it uses the document label
              all_labels.append(row['Paragraph' + str(i)] if row['Paragraph' + str(i)] >= 0 else text_label)

      tokens = pad_sequence(tokens, batch_first=True)
      labels = torch.tensor(all_labels)
      dataset = TensorDataset(tokens, labels)
      return dataset

  def get_data_loaders(self, batch_size=32, shuffle=True):
      processed_dataset = self.tokenize_data()

      data_loader = DataLoader(
          processed_dataset,
          shuffle=shuffle,
          batch_size=batch_size
      )
      return data_loader


The model's fine-tuning stages by giving it the task specific data.

The Optimizer is created in the execution stage, which have the hyperparameters of learning rate and epsilon.

In [ ]:
class Trainer():

  def __init__(self, options):
    self.device = options['device']
    self.train_data = options['train_data']
    self.batch_size = options['batch_size']
    self.epochs = options['epochs']
    self.save_path = options['save_path']
    transformer = DistillBERT()
    self.model, self.tokenizer = transformer.get_tokenizer_and_model()
    self.model.to(self.device)

  def get_performance_metrics(self, preds, labels):
    pred_flat = np.argmax(preds, axis=1).flatten()
    labels_flat = labels.flatten()
    precision = precision_score(labels_flat, pred_flat, zero_division=0, average='weighted')
    recall = recall_score(labels_flat, pred_flat, zero_division=0, average='weighted')
    f1 = f1_score(labels_flat, pred_flat, zero_division=0, average='weighted')
    return precision, recall, f1

  def set_training_parameters(self):
    for i, layer in enumerate(self.model.distilbert.transformer.layer):
      for param in layer.parameters():
        param.requires_grad = True

    for param in self.model.classifier.parameters():
      param.requires_grad = True

  def train(self, data_loader, optimizer):
    self.model.train()
    total_recall = 0
    total_precision = 0
    total_f1 = 0
    total_loss = 0

    for batch_idx, (text, labels) in enumerate(tqdm(data_loader)):
      self.model.zero_grad()

      text = text.to(self.device)
      labels = labels.to(self.device)

      outputs = self.model(text, labels=labels)
      predicted = outputs.logits.cpu().detach().numpy()

      loss = outputs.loss
      total_loss += loss.item()
      loss.backward()
      optimizer.step()

      precision, recall, f1 = self.get_performance_metrics(predicted, labels.cpu().numpy())
      total_precision += precision
      total_recall += recall
      total_f1 += f1

    precision = total_precision/len(data_loader)
    recall = total_recall/len(data_loader)
    f1 = total_f1/len(data_loader)
    loss = total_loss/len(data_loader)

    return precision, recall, f1, loss

  def eval(self, data_loader):
    self.model.eval()
    total_recall = 0
    total_precision = 0
    total_f1 = 0
    total_loss = 0

    with torch.no_grad():
      for batch_idx, (text, labels) in enumerate(tqdm(data_loader)):
        text = text.to(self.device)
        labels = labels.to(self.device)

        outputs = self.model(text, labels=labels)
        predicted = outputs.logits.cpu().detach().numpy()

        loss = outputs.loss
        total_loss += loss.item()

        precision, recall, f1 = self.get_performance_metrics(predicted, labels.cpu().numpy())
        total_precision += precision
        total_recall += recall
        total_f1 += f1

      precision = total_precision/len(data_loader)
      recall = total_recall/len(data_loader)
      f1 = total_f1/len(data_loader)
      loss = total_loss/len(data_loader)

      return precision, recall, f1, loss

  def save_transformer(self):
    self.model.save_pretrained(self.save_path)
    self.tokenizer.save_pretrained(self.save_path)

  def execute(self):
    last_best = 0
    train_dataset = ParagraphDatasetLoader(self.train_data, self.tokenizer)
    train_data_loader = train_dataset.get_data_loaders(self.batch_size)

    val_dataset = ParagraphDatasetLoader(validation_dataset, self.tokenizer)
    val_data_loader = val_dataset.get_data_loaders(self.batch_size)

    optimizer = torch.optim.AdamW(self.model.parameters(), lr = 3e-5, eps = 1e-8)
    self.set_training_parameters()

    for epoch_i in range(0, self.epochs):
      train_precision, train_recall, train_f1, train_loss = self.train(train_data_loader, optimizer)
      print(f'Epoch {epoch_i + 1}: train_loss: {train_loss:.4f} train_precision: {train_precision:.4f} train_recall: {train_recall:.4f} train_f1: {train_f1:.4f}')
      val_precision, val_recall, val_f1, val_loss = self.eval(val_data_loader)
      print(f'Epoch {epoch_i + 1}: val_loss: {val_loss:.4f} val_precision: {val_precision:.4f} val_recall: {val_recall:.4f} val_f1: {val_f1:.4f}')

      if val_f1 > last_best:
        print("Saving model..")
        self.save_transformer()
        last_best = val_f1
        print("Model saved.")

Hyper Params of
1. Batch size
2. Epochs

Then the additional of save path of the model which is set to *distilbert_per_sent_idea_2* at the moment

## Idea 2.1 Training

training on the train_dataset we've loaded up previously, then verfying it with the validation_dataset each epochs to see the improvement of the model

In [ ]:
BATCH_SIZE = 16
EPOCHS = 3

SAVE_PATH = 'distilbert_per_sent_idea_2.1'

In [ ]:
options = {}
options['device'] = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
options['train_data'] = train_dataset
options['batch_size'] = BATCH_SIZE
options['epochs'] = EPOCHS
options['save_path'] = SAVE_PATH

trainer = Trainer(options)
trainer.execute()

/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:797: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/483 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

...

Epoch 2: val_loss: 0.9116 val_precision: 0.5539 val_recall: 0.5364 val_f1: 0.5239
Saving model..
Model saved.


100%|██████████| 1872/1872 [25:06<00:00,  1.24it/s]


Epoch 3: train_loss: 0.6853 train_precision: 0.7241 train_recall: 0.6892 train_f1: 0.6826


100%|██████████| 306/306 [01:24<00:00,  3.61it/s]

Epoch 3: val_loss: 0.9974 val_precision: 0.5593 val_recall: 0.5153 val_f1: 0.5131


## Idea 2.1 Testing

In [ ]:
class Tester:
  def __init__(self, options):
    self.device = options['device']
    self.test_data = options['test_data']
    self.batch_size = options['batch_size']
    self.save_path = options['save_path']

    transformer = DistillBERT()
    self.model, self.tokenizer = transformer.get_tokenizer_and_model()
    self.model.to(self.device)

  def load_transformer(self):
    self.model = AutoModelForSequenceClassification.from_pretrained(self.save_path)
    self.tokenizer = AutoTokenizer.from_pretrained(self.save_path)
    self.model.to(self.device)
    self.model.eval()

  def get_performance_metrics(self, preds, labels):
    pred_flat = np.argmax(preds, axis=1).flatten()
    labels_flat = labels.flatten()
    precision = precision_score(labels_flat, pred_flat, zero_division=0, average='weighted')
    recall = recall_score(labels_flat, pred_flat, zero_division=0, average='weighted')
    f1 = f1_score(labels_flat, pred_flat, zero_division=0, average='weighted')
    return precision, recall, f1

  def test(self, data_loader):
    self.model.eval()
    total_recall = 0
    total_precision = 0
    total_f1 = 0
    total_loss = 0

    y_true = []
    y_pred = []

    with torch.no_grad():
      for batch_idx, (text, labels) in enumerate(tqdm(data_loader)):
        text = text.to(self.device)
        labels = labels.to(self.device)

        outputs = self.model(text, labels=labels)
        predicted = outputs.logits.cpu().detach().numpy()
        loss = outputs.loss
        total_loss += loss.item()

        labels = labels.cpu().numpy()
        y_true.extend(labels)
        y_pred.extend(np.argmax(predicted, axis=1))

        precision, recall, f1 = self.get_performance_metrics(predicted, labels)
        total_precision += precision
        total_recall += recall
        total_f1 += f1

      precision = total_precision / len(data_loader)
      recall = total_recall / len(data_loader)
      f1 = total_f1 / len(data_loader)
      loss = total_loss / len(data_loader)
      report = classification_report(y_true, y_pred, target_names=["Negative", "Neutral", "Positive"])

      return precision, recall, f1, loss, report

  def execute(self):
    self.load_transformer()
    test_dataset = DatasetLoader(self.test_data, self.tokenizer)
    test_data_loader = test_dataset.get_data_loaders(self.batch_size)
    precision, recall, f1, loss, report = self.test(test_data_loader)
    print(f'precision: {precision:.4f} recall: {recall:.4f} f1: {f1:.4f}')
    print(f'Test loss: {loss:.4f}')
    print()
    print(report)


In [ ]:
options={}
options['device'] = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
options['test_data'] = test_random_dataset
options['batch_size'] = BATCH_SIZE
options['save_path'] = SAVE_PATH

tester = Tester(options)
tester.execute()

/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:797: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/483 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

...

model.safetensors:   0%|          | 0.00/268M [00:00<?, ?B/s]

Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight', 'pre_classifier.bias', 'pre_classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Tokenizing data...


100%|██████████| 37/37 [10:51<00:00, 17.61s/it]

precision: 0.5568 recall: 0.5873 f1: 0.5597
Test loss: 0.8639

              precision    recall  f1-score   support

    Negative       0.50      0.05      0.10        73
     Neutral       0.49      0.56      0.52       213
    Positive       0.66      0.73      0.70       293

    accuracy                           0.59       579
   macro avg       0.55      0.45      0.44       579
weighted avg       0.58      0.59      0.56       579



In [ ]:
options={}
options['device'] = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
options['test_data'] = test_fixed_dataset
options['batch_size'] = BATCH_SIZE
options['save_path'] = SAVE_PATH

tester = Tester(options)
tester.execute()

/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:797: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight', 'pre_classifier.bias', 'pre_classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Tokenizing data...


100%|██████████| 52/52 [14:59<00:00, 17.29s/it]

precision: 0.4410 recall: 0.4786 f1: 0.4442
Test loss: 1.0437

              precision    recall  f1-score   support

    Negative       0.42      0.04      0.07       139
     Neutral       0.43      0.52      0.47       320
    Positive       0.52      0.61      0.56       368

    accuracy                           0.48       827
   macro avg       0.46      0.39      0.37       827
weighted avg       0.47      0.48      0.44       827



## Idea 2.2 Training

In [ ]:
class ParagraphDatasetLoader(Dataset):
##Redefining ParagraphDatasetLoader with paragraph only
  def __init__(self, dataset, tokenizer):
      self.dataset = dataset
      self.tokenizer = tokenizer

  def tokenize_data(self):
      print("Tokenizing data...")
      tokens = []
      paragraphs = []
      all_labels = []
      for row in tqdm(self.dataset):
        #1st, 2nd document in str
        text = row['MASKED_DOCUMENT']
        #true sentimenet of whole articles
        text_label = row['TRUE_SENTIMENT']

        paragraph_labels = []
        labels = [label for label in row.keys() if label.startswith('Paragraph')]
        paragraphs = text.split('\n')
        for i,paragraph in zip(range(len(paragraphs)),paragraphs):
          tokens.append(torch.tensor(self.tokenizer.encode(paragraph, max_length=512, truncation=True, add_special_tokens=True)))
          #surpass 15 paragraphs use the  document label
          if i > 15:
            all_labels.append(text_label)
          else:
            #if there's a paragraph label(0,1,2) assign that to corresponding paragraph, ow it uses the document label
             all_labels.append(row['Paragraph' + str(i)] if row['Paragraph' + str(i)] >= 0 else text_label)

      tokens = pad_sequence(tokens, batch_first=True)
      labels = torch.tensor(all_labels)
      dataset = TensorDataset(tokens, labels)
      return dataset

  def get_data_loaders(self, batch_size=32,  shuffle=True):
      processed_dataset = self.tokenize_data()

      data_loader = DataLoader(
          processed_dataset,
          shuffle=shuffle,
          batch_size=batch_size
      )
      return data_loader


In [ ]:
class Trainer():

  def __init__(self, options):
    self.device = options['device']
    self.train_data = options['train_data']
    self.batch_size = options['batch_size']
    self.epochs = options['epochs']
    self.save_path = options['save_path']
    transformer = DistillBERT()
    self.model, self.tokenizer = transformer.get_tokenizer_and_model()
    self.model.to(self.device)

  def get_performance_metrics(self, preds, labels):
    pred_flat = np.argmax(preds, axis=1).flatten()
    labels_flat = labels.flatten()
    precision = precision_score(labels_flat, pred_flat, zero_division=0, average='weighted')
    recall = recall_score(labels_flat, pred_flat, zero_division=0, average='weighted')
    f1 = f1_score(labels_flat, pred_flat, zero_division=0, average='weighted')
    return precision, recall, f1

  def set_training_parameters(self):
    for i, layer in enumerate(self.model.distilbert.transformer.layer):
      for param in layer.parameters():
        param.requires_grad = True

    for param in self.model.classifier.parameters():
      param.requires_grad = True

  def train(self, data_loader, optimizer):
    self.model.train()
    total_recall = 0
    total_precision = 0
    total_f1 = 0
    total_loss = 0

    for batch_idx, (text, labels) in enumerate(tqdm(data_loader)):
      self.model.zero_grad()

      text = text.to(self.device)
      labels = labels.to(self.device)

      outputs = self.model(text, labels=labels)
      predicted = outputs.logits.cpu().detach().numpy()

      loss = outputs.loss
      total_loss += loss.item()
      loss.backward()
      optimizer.step()

      precision, recall, f1 = self.get_performance_metrics(predicted, labels.cpu().numpy())
      total_precision += precision
      total_recall += recall
      total_f1 += f1

    precision = total_precision/len(data_loader)
    recall = total_recall/len(data_loader)
    f1 = total_f1/len(data_loader)
    loss = total_loss/len(data_loader)

    return precision, recall, f1, loss

  def eval(self, data_loader):
    self.model.eval()
    total_recall = 0
    total_precision = 0
    total_f1 = 0
    total_loss = 0

    with torch.no_grad():
      for batch_idx, (text, labels) in enumerate(tqdm(data_loader)):
        text = text.to(self.device)
        labels = labels.to(self.device)

        outputs = self.model(text, labels=labels)
        predicted = outputs.logits.cpu().detach().numpy()

        loss = outputs.loss
        total_loss += loss.item()

        precision, recall, f1 = self.get_performance_metrics(predicted, labels.cpu().numpy())
        total_precision += precision
        total_recall += recall
        total_f1 += f1

      precision = total_precision/len(data_loader)
      recall = total_recall/len(data_loader)
      f1 = total_f1/len(data_loader)
      loss = total_loss/len(data_loader)

      return precision, recall, f1, loss

  def save_transformer(self):
    self.model.save_pretrained(self.save_path)
    self.tokenizer.save_pretrained(self.save_path)

  def execute(self):
    last_best = 0
    paragraph_dataset = ParagraphDatasetLoader(self.train_data, self.tokenizer)
    paragraph_data_loader = paragraph_dataset.get_data_loaders(self.batch_size)
    train_dataset = DatasetLoader(self.train_data, self.tokenizer)
    train_data_loader = train_dataset.get_data_loaders(self.batch_size)

    paragraph_val_dataset = ParagraphDatasetLoader(validation_dataset, self.tokenizer)
    paragraph_val_data_loader = paragraph_val_dataset.get_data_loaders(self.batch_size)
    val_dataset = DatasetLoader(validation_dataset, self.tokenizer)
    val_data_loader = val_dataset.get_data_loaders(self.batch_size)

    optimizer = torch.optim.AdamW(self.model.parameters(), lr = 3e-5, eps = 1e-8)
    self.set_training_parameters()

    for epoch_i in range(0, self.epochs):
      train_precision, train_recall, train_f1, train_loss = self.train(paragraph_data_loader, optimizer)
      print(f'Epoch {epoch_i + 1}: train_loss: {train_loss:.4f} train_precision: {train_precision:.4f} train_recall: {train_recall:.4f} train_f1: {train_f1:.4f}')
      val_precision, val_recall, val_f1, val_loss = self.eval(paragraph_val_data_loader)
      print(f'Epoch {epoch_i + 1}: val_loss: {val_loss:.4f} val_precision: {val_precision:.4f} val_recall: {val_recall:.4f} val_f1: {val_f1:.4f}')
    for epoch_i in range(0, self.epochs):
      train_precision, train_recall, train_f1, train_loss = self.train(train_data_loader, optimizer)
      print(f'Epoch {epoch_i + 1}: train_loss: {train_loss:.4f} train_precision: {train_precision:.4f} train_recall: {train_recall:.4f} train_f1: {train_f1:.4f}')
      val_precision, val_recall, val_f1, val_loss = self.eval(val_data_loader)
      print(f'Epoch {epoch_i + 1}: val_loss: {val_loss:.4f} val_precision: {val_precision:.4f} val_recall: {val_recall:.4f} val_f1: {val_f1:.4f}')

      if val_f1 > last_best:
        print("Saving model..")
        self.save_transformer()
        last_best = val_f1
        print("Model saved.")

In [ ]:
BATCH_SIZE = 16
EPOCHS = 3

SAVE_PATH = 'distilbert_per_sent_idea_2.2'

In [ ]:
options = {}
options['device'] = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
options['train_data'] = train_dataset
options['batch_size'] = BATCH_SIZE
options['epochs'] = EPOCHS
options['save_path'] = SAVE_PATH

trainer = Trainer(options)
trainer.execute()

## Idea 2.2 Testing

Verify the model performance with fresh data from testing split

In [ ]:
options={}
options['device'] = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
options['test_data'] = test_random_dataset
options['batch_size'] = BATCH_SIZE
options['save_path'] = SAVE_PATH

tester = Tester(options)
tester.execute()

Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight', 'pre_classifier.bias', 'pre_classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Tokenizing data...


100%|██████████| 37/37 [10:54<00:00, 17.70s/it]

precision: 0.5426 recall: 0.5394 f1: 0.5216
Test loss: 0.9504

              precision    recall  f1-score   support

    Negative       0.27      0.04      0.07        73
     Neutral       0.45      0.61      0.52       213
    Positive       0.65      0.62      0.64       293

    accuracy                           0.54       579
   macro avg       0.46      0.42      0.41       579
weighted avg       0.53      0.54      0.52       579



In [ ]:
options={}
options['device'] = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
options['test_data'] = test_fixed_dataset
options['batch_size'] = BATCH_SIZE
options['save_path'] = SAVE_PATH

tester = Tester(options)
tester.execute()

/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:797: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight', 'pre_classifier.bias', 'pre_classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Tokenizing data...


100%|██████████| 52/52 [15:33<00:00, 17.96s/it]

precision: 0.4594 recall: 0.4714 f1: 0.4393
Test loss: 1.0884

              precision    recall  f1-score   support

    Negative       0.47      0.05      0.09       139
     Neutral       0.42      0.61      0.50       320
    Positive       0.53      0.51      0.52       368

    accuracy                           0.47       827
   macro avg       0.47      0.39      0.37       827
weighted avg       0.48      0.47      0.44       827



## Idea 3 Weighted Class

Using DistillBERT with 3 classes with their weight distribution, and a special mask_token which is represented by [TGT] in Masked document of the dataset

In [ ]:
class DistillBERT():
  def __init__(self, model_name='distilbert-base-uncased', num_classes=3):
    self.tokenizer = AutoTokenizer.from_pretrained(model_name)
    self.tokenizer.add_special_tokens({'mask_token': '[TGT]'})
    self.model = AutoModelForSequenceClassification.from_pretrained(model_name, num_labels=num_classes)
    self.model.resize_token_embeddings(len(self.tokenizer))


  def get_tokenizer_and_model(self):
    return self.model, self.tokenizer

Using the model's tokenizer to tokenize the input dataset

In [ ]:
class DatasetLoader(Dataset):

  def __init__(self, dataset, tokenizer):
    self.dataset = dataset
    self.tokenizer = tokenizer

  def tokenize_data(self):
    print("Tokenizing data...")
    tokens = []
    labels = []

    positive_count = 0
    neutral_count = 0
    negative_count = 0

    for row in tqdm(self.dataset):
      text = row['MASKED_DOCUMENT']
      label = row['TRUE_SENTIMENT']

      tokens.append(torch.tensor(self.tokenizer.encode(text, max_length=512, truncation=True, add_special_tokens=True)))

      if label == 0:
        negative_count += 1
      elif label == 1:
        neutral_count += 1
      else:
        positive_count += 1
      labels.append(label)


    tokens = pad_sequence(tokens, batch_first=True)
    labels = torch.tensor(labels)
    dataset = TensorDataset(tokens, labels)
    return dataset, positive_count, neutral_count, negative_count

  def get_data_loaders(self, batch_size=32, shuffle=True):
    processed_dataset, positive_count, neutral_count, negative_count = self.tokenize_data()

    data_loader = DataLoader(
        processed_dataset,
        shuffle=shuffle,
        batch_size=batch_size
    )

    return data_loader, positive_count, neutral_count, negative_count

The model's fine-tuning stages by giving it the task specific data.

The Optimizer is created in the execution stage, which have the hyperparameters of learning rate and epsilon.

The model will use a weight to calculate's its weight

In [ ]:
class Trainer():

  def __init__(self, options):
    self.device = options['device']
    self.train_data = options['train_data']
    self.batch_size = options['batch_size']
    self.epochs = options['epochs']
    self.save_path = options['save_path']
    transformer = DistillBERT()
    self.model, self.tokenizer = transformer.get_tokenizer_and_model()
    self.model.to(self.device)

  def get_performance_metrics(self, preds, labels):
    pred_flat = np.argmax(preds, axis=1).flatten()
    labels_flat = labels.flatten()
    precision = precision_score(labels_flat, pred_flat, zero_division=0, average='weighted')
    recall = recall_score(labels_flat, pred_flat, zero_division=0, average='weighted')
    f1 = f1_score(labels_flat, pred_flat, zero_division=0, average='weighted')
    return precision, recall, f1

  def set_training_parameters(self):
    for i, layer in enumerate(self.model.distilbert.transformer.layer):
      for param in layer.parameters():
        param.requires_grad = True

    for param in self.model.classifier.parameters():
      param.requires_grad = True

  def train(self, data_loader, optimizer, positive_count, neutral_count, negative_count):
    self.model.train()
    total_recall = 0
    total_precision = 0
    total_f1 = 0
    total_loss = 0

    total_data = positive_count + neutral_count + negative_count
    positive_weight = total_data / (3 * positive_count)
    neutral_weight = total_data / (3 * neutral_count)
    negative_weight = total_data / (3 * negative_count)

    class_weights = torch.tensor([negative_weight, neutral_weight, positive_weight]).to(self.device)

    for batch_idx, (text, labels) in enumerate(tqdm(data_loader)):
      self.model.zero_grad()

      text = text.to(self.device)
      labels = labels.to(self.device)

      outputs = self.model(text, labels=labels)
      logits = outputs.logits
      criterion = CrossEntropyLoss(weight=class_weights)
      loss = criterion(logits, labels)

      predicted = outputs.logits.cpu().detach().numpy()

      total_loss += loss.item()
      loss.backward()
      optimizer.step()

      precision, recall, f1 = self.get_performance_metrics(predicted, labels.cpu().numpy())
      total_precision += precision
      total_recall += recall
      total_f1 += f1

    precision = total_precision/len(data_loader)
    recall = total_recall/len(data_loader)
    f1 = total_f1/len(data_loader)
    loss = total_loss/len(data_loader)

    return precision, recall, f1, loss

  def eval(self, data_loader, positive_count, neutral_count, negative_count):
    self.model.eval()
    total_recall = 0
    total_precision = 0
    total_f1 = 0
    total_loss = 0

    total_data = positive_count + neutral_count + negative_count
    positive_weight = total_data / (3 * positive_count)
    neutral_weight = total_data / (3 * neutral_count)
    negative_weight = total_data / (3 * negative_count)

    class_weights = torch.tensor([negative_weight, neutral_weight, positive_weight]).to(self.device)

    with torch.no_grad():
      for batch_idx, (text, labels) in enumerate(tqdm(data_loader)):
        text = text.to(self.device)
        labels = labels.to(self.device)

        outputs = self.model(text, labels=labels)
        logits = outputs.logits
        criterion = CrossEntropyLoss(weight=class_weights)
        loss = criterion(logits, labels)

        predicted = outputs.logits.cpu().detach().numpy()

        total_loss += loss.item()

        precision, recall, f1 = self.get_performance_metrics(predicted, labels.cpu().numpy())
        total_precision += precision
        total_recall += recall
        total_f1 += f1

      precision = total_precision/len(data_loader)
      recall = total_recall/len(data_loader)
      f1 = total_f1/len(data_loader)
      loss = total_loss/len(data_loader)

      return precision, recall, f1, loss

  def save_transformer(self):
    self.model.save_pretrained(self.save_path)
    self.tokenizer.save_pretrained(self.save_path)

  def execute(self):
    last_best = 0
    train_dataset = DatasetLoader(self.train_data, self.tokenizer)
    train_data_loader, train_positive_count, train_neutral_count, train_negative_count = train_dataset.get_data_loaders(self.batch_size)

    val_dataset = DatasetLoader(validation_dataset, self.tokenizer)
    val_data_loader, val_positive_count, val_neutral_count, val_negative_count = val_dataset.get_data_loaders(self.batch_size)

    optimizer = torch.optim.AdamW(self.model.parameters(), lr = 3e-5, eps = 1e-8)
    self.set_training_parameters()

    for epoch_i in range(0, self.epochs):
      train_precision, train_recall, train_f1, train_loss = self.train(train_data_loader, optimizer, train_positive_count, train_neutral_count, train_negative_count)
      print(f'Epoch {epoch_i + 1}: train_loss: {train_loss:.4f} train_precision: {train_precision:.4f} train_recall: {train_recall:.4f} train_f1: {train_f1:.4f}')
      val_precision, val_recall, val_f1, val_loss = self.eval(val_data_loader, val_positive_count, val_neutral_count, val_negative_count)
      print(f'Epoch {epoch_i + 1}: val_loss: {val_loss:.4f} val_precision: {val_precision:.4f} val_recall: {val_recall:.4f} val_f1: {val_f1:.4f}')

      if val_f1 > last_best:
        print("Saving model..")
        self.save_transformer()
        last_best = val_f1
        print("Model saved.")

Hyper Params of
1. Batch size
2. Epochs

Then the additional of save path of the model which is set to *distilbert_per_sent_baseline* at the moment

In [ ]:
BATCH_SIZE = 16
EPOCHS = 3

SAVE_PATH = 'distilbert_per_sent_idea_3'

## Idea 3 Training
training on the train_dataset we've loaded up previously, then verfying it with the validation_dataset each epochs to see the improvement of the model

In [ ]:
options = {}
options['device'] = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
options['train_data'] = train_dataset
options['batch_size'] = BATCH_SIZE
options['epochs'] = EPOCHS
options['save_path'] = SAVE_PATH

trainer = Trainer(options)
trainer.execute()

/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:797: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight', 'pre_classifier.bias', 'pre_classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Tokenizing data...


100%|██████████| 3355/3355 [00:08<00:00, 375.27it/s]


Tokenizing data...


100%|██████████| 210/210 [02:45<00:00,  1.27it/s]


...

Epoch 2: val_loss: 0.9606 val_precision: 0.6089 val_recall: 0.4459 val_f1: 0.4720


100%|██████████| 210/210 [02:46<00:00,  1.26it/s]


Epoch 3: train_loss: 0.8709 train_precision: 0.6486 train_recall: 0.5513 train_f1: 0.5601


100%|██████████| 37/37 [00:10<00:00,  3.70it/s]

Epoch 3: val_loss: 0.9534 val_precision: 0.6420 val_recall: 0.4645 val_f1: 0.4930


## Idea 3 Testing
Verifying the model performance with fresh data from testing split.

In [ ]:
class Tester:
  def __init__(self, options):
    self.device = options['device']
    self.test_data = options['test_data']
    self.batch_size = options['batch_size']
    self.save_path = options['save_path']

    transformer = DistillBERT()
    self.model, self.tokenizer = transformer.get_tokenizer_and_model()
    self.model.to(self.device)

  def load_transformer(self):
    self.model = AutoModelForSequenceClassification.from_pretrained(self.save_path)
    self.tokenizer = AutoTokenizer.from_pretrained(self.save_path)
    self.model.to(self.device)
    self.model.eval()

  def get_performance_metrics(self, preds, labels):
    pred_flat = np.argmax(preds, axis=1).flatten()
    labels_flat = labels.flatten()
    precision = precision_score(labels_flat, pred_flat, zero_division=0, average='weighted')
    recall = recall_score(labels_flat, pred_flat, zero_division=0, average='weighted')
    f1 = f1_score(labels_flat, pred_flat, zero_division=0, average='weighted')
    return precision, recall, f1

  def test(self, data_loader, positive_count, neutral_count, negative_count):
    self.model.eval()
    total_recall = 0
    total_precision = 0
    total_f1 = 0
    total_loss = 0

    y_true = []
    y_pred = []

    total_data = positive_count + neutral_count + negative_count
    positive_weight = total_data / (3 * positive_count)
    neutral_weight = total_data / (3 * neutral_count)
    negative_weight = total_data / (3 * negative_count)

    class_weights = torch.tensor([negative_weight, neutral_weight, positive_weight]).to(self.device)
    print(class_weights)

    with torch.no_grad():
      for batch_idx, (text, labels) in enumerate(tqdm(data_loader)):
        text = text.to(self.device)
        labels = labels.to(self.device)

        outputs = self.model(text, labels=labels)
        logits = outputs.logits
        criterion = CrossEntropyLoss(weight=class_weights)
        loss = criterion(logits, labels)

        predicted = outputs.logits.cpu().detach().numpy()

        total_loss += loss.item()

        labels = labels.cpu().numpy()


        y_true.extend(labels)
        y_pred.extend(np.argmax(predicted, axis=1))

        precision, recall, f1 = self.get_performance_metrics(predicted, labels)
        total_precision += precision
        total_recall += recall
        total_f1 += f1

      precision = total_precision / len(data_loader)
      recall = total_recall / len(data_loader)
      f1 = total_f1 / len(data_loader)
      loss = total_loss / len(data_loader)
      report = classification_report(y_true, y_pred, target_names=["Negative", "Neutral", "Positive"])

      return precision, recall, f1, loss, report

  def execute(self):
    self.load_transformer()
    test_dataset = DatasetLoader(self.test_data, self.tokenizer)
    test_data_loader, test_positive_count, test_neutral_count, test_negative_count = test_dataset.get_data_loaders(self.batch_size)
    precision, recall, f1, loss, report = self.test(test_data_loader, test_positive_count, test_neutral_count, test_negative_count)
    print(f'precision: {precision:.4f} recall: {recall:.4f} f1: {f1:.4f}')
    print(f'Test loss: {loss:.4f}')
    print()
    print(report)


Testing on the test_random_dataset (random and different main target entity)

In [ ]:
options={}
options['device'] = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
options['test_data'] = test_random_dataset
options['batch_size'] = BATCH_SIZE
options['save_path'] = SAVE_PATH

tester = Tester(options)
tester.execute()

/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:797: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight', 'pre_classifier.bias', 'pre_classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Tokenizing data...


100%|██████████| 579/579 [00:01<00:00, 561.30it/s]


tensor([2.6438, 0.9061, 0.6587], device='cuda:0')


100%|██████████| 37/37 [00:09<00:00,  3.76it/s]

precision: 0.5475 recall: 0.4482 f1: 0.4609
Test loss: 1.0173

              precision    recall  f1-score   support

    Negative       0.22      0.67      0.34        73
     Neutral       0.40      0.21      0.28       213
    Positive       0.67      0.57      0.62       293

    accuracy                           0.45       579
   macro avg       0.43      0.48      0.41       579
weighted avg       0.52      0.45      0.46       579



Testing on the test_fixed_dataset (stuff have repeat high frequence main target entity)

In [ ]:
options={}
options['device'] = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
options['test_data'] = test_fixed_dataset
options['batch_size'] = BATCH_SIZE
options['save_path'] = SAVE_PATH

tester = Tester(options)
tester.execute()

/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:797: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight', 'pre_classifier.bias', 'pre_classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Tokenizing data...


100%|██████████| 827/827 [00:01<00:00, 549.11it/s]


tensor([1.9832, 0.8615, 0.7491], device='cuda:0')


100%|██████████| 52/52 [00:14<00:00,  3.57it/s]

precision: 0.4598 recall: 0.3760 f1: 0.3881
Test loss: 1.1028

              precision    recall  f1-score   support

    Negative       0.21      0.46      0.28       139
     Neutral       0.40      0.30      0.34       320
    Positive       0.55      0.41      0.47       368

    accuracy                           0.38       827
   macro avg       0.38      0.39      0.37       827
weighted avg       0.43      0.38      0.39       827

